# **Validación Numérica del QNM (Modo 22) + Corrección EDR**
Repositorio: *EDR-Validation*
Autor: **Camilo Robinson**

Este notebook reproduce numéricamente:
- El potencial perturbativo EDR δV(r)
- El modo base GR ψ₀(r)
- La integral ∫ ψ₀ δV ψ₀ dr
- El corrimiento de frecuencia δω
- El parámetro α_flow
- La predicción del corrimiento en Hz

Este es uno de los pilares del artículo científico.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from edr_alpha import compute_alpha_flow
from edr_deltaV import deltaV
from edr_modes import psi0
from edr_kerr_geometry import r_plus

## **1. Parámetros del agujero negro**
Puedes modificarlos libremente para reproducir diferentes eventos LIGO/Virgo.

In [ ]:
M_solar = 60  # masa en M_sun
a = 0.72      # parámetro de espín
omega0 = 0.52 # frecuencia base GR geométrica

## **2. Gráfica del potencial δV(r)**

In [ ]:
M = 1
rH = r_plus(M, a)

r = np.linspace(rH+0.01, 100, 5000)
dV = deltaV(r, rH)

plt.plot(r, dV)
plt.xlabel('r')
plt.ylabel('δV')
plt.title('Potencial EDR δV(r)')
plt.grid()
plt.show()

## **3. Gráfica del modo base ψ₀(r)**

In [ ]:
psi = psi0(r, rH)
plt.plot(r, psi)
plt.title('Modo Base ψ₀(r)')
plt.xlabel('r')
plt.ylabel('ψ₀')
plt.grid(True)
plt.show()

## **4. Cálculo del parámetro α_flow y corrimiento en Hz**

In [ ]:
d_omega, alpha, df_Hz = compute_alpha_flow(M_solar, a, omega0)

print('δω geom    =', d_omega)
print('α_flow     =', alpha)
print('Δf (Hz)    =', df_Hz)

## **Conclusión**
Este notebook permite validar que la teoría EDR genera:
- un potencial perturbativo bien definido
- un modo base estable
- y un corrimiento consistente con un término físico δV.

Guarda este notebook como evidencia computacional.